<a href="https://colab.research.google.com/github/jae-finger/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module3-join-and-reshape-data/LS_DS_113_Join_and_Reshape_Data_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-11-06 02:28:08--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.88.26
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.88.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’

instacart_online_gr 100%[===================>] 196.03M  81.1MB/s    in 2.4s    

2019-11-06 02:28:10 (81.1 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’ saved [205548478/205548478]



In [0]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [0]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [0]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [0]:
#Import the key players
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
#Read in the products csv. It will probably be important to have a list of product names with ids
products_raw = pd.read_csv('products.csv')
products_raw.head(3)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7


In [0]:
#Minor adjusting to the frame, reducing features likely not needed
products_clean = products_raw[['product_id', 'product_name']]
print(products_clean.shape)
products_clean.head(5)

(49688, 2)


,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce


In [0]:
#Fix the training testing sets by concatenating them
order_products__prior = pd.read_csv('order_products__prior.csv')
order_products__train = pd.read_csv('order_products__train.csv')
ordered_products = pd.concat([order_products__prior, order_products__train])
print(ordered_products.shape)
ordered_products.head(5)

(33819106, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [0]:
#Reducing the number of features in this dataframe
ordered_products_clean = ordered_products[['order_id', 'product_id']]
print(ordered_products_clean.shape)
ordered_products_clean.head(5)

(33819106, 2)


,order_id,product_id
0,2,33120
1,2,28985
2,2,9327
3,2,45918
4,2,30035


In [0]:
#Join the two datasets so I can see a list of ordered products with their names minus extra features
products_ordered = pd.merge(ordered_products_clean, products_clean, how='left', on='product_id')
print(products_ordered.shape)
products_ordered.head(4)

(33819106, 3)


,order_id,product_id,product_name
0,2,33120,Organic Egg Whites
1,2,28985,Michigan Organic Kale
2,2,9327,Garlic Powder
3,2,45918,Coconut Butter


In [0]:
#Now I just need to filter/group by product name and how many times each one is there.
numbers_of_products = products_ordered['product_name'].value_counts()

In [0]:
#To finish we can limit it to the top 10
numbers_of_products.head(10)

Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: product_name, dtype: int64

## Reshape Data Section

- Replicate the lesson code
- Complete the code cells we skipped near the beginning of the notebook
- Table 2 --> Tidy
- Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

####Replicate lesson code to create table 2

In [0]:
#Recreate table 1 so it can be transposed
table1 = pd.DataFrame([[np.nan, 2],
          [16, 11],
          [3, 1],
          ], index=['John Smith', 'Jane Doe', 'Mary Johnson'], columns=['treatmenta', 'treatmentb'])

#Transpose table 1 to table 2
table2 = table1.T
table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


#### table2 -> tidy

In [0]:
#Get the columns in a list
table2.columns.tolist()

['John Smith', 'Jane Doe', 'Mary Johnson']

In [0]:
#Next get the index in a list
table2.index.tolist() 

['treatmenta', 'treatmentb']

In [0]:
#Reset the index
table2 = table2.reset_index()
table2

,index,John Smith,Jane Doe,Mary Johnson
0,treatmenta,NaN,16.0,3.0
1,treatmentb,2.0,11.0,1.0


In [0]:
#Time to tidy
tidy = table2.melt(id_vars='index', value_vars=['John Smith', 'Jane Doe', 'Mary Johnson'])
tidy

,index,variable,value
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


#### tidy -> table2

In [0]:
#Reverse tidy back to table2
wide = tidy.pivot_table(values='value', index='index', columns='variable')
wide

variable,Jane Doe,John Smith,Mary Johnson
index,,,
treatmenta,16.0,NaN,3.0
treatmentb,11.0,2.0,1.0


####flights - Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [0]:
flights = sns.load_dataset('flights')

In [0]:
#Examine the dataset
print(flights.shape)
flights.head(4)

(144, 3)


,year,month,passengers
0,1949,January,112
1,1949,February,118
2,1949,March,132
3,1949,April,129


In [0]:
#Construct pivot table and check January & December values for match
my_pivot_table = flights.pivot_table(values='passengers', index='year', columns='month')
print(my_pivot_table.head(1))
print(my_pivot_table.tail(1))


month  January  February  March  April  ...  September  October  November  December
year                                    ...                                        
1949       112       118    132    129  ...        136      119       104       118

[1 rows x 12 columns]
month  January  February  March  April  ...  September  October  November  December
year                                    ...                                        
1960       417       391    419    461  ...        508      461       390       432

[1 rows x 12 columns]


## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [0]:
#Sketched out the basic idea of product id -> product name -> order hour . Also must filter based on popularity
#Start out with data I almost assembled already
products_ordered.tail(3)

,order_id,product_id,product_name
33819103,3421070,35951,Organic Unsweetened Almond Milk
33819104,3421070,16953,Creamy Peanut Butter
33819105,3421070,4724,Broccoli Florettes


In [0]:
#Find order times
orders = pd.read_csv('orders.csv')
orders.head(3)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0


In [0]:
#Reduce orders
orders_clean = orders[['order_id', 'order_hour_of_day']]
orders_clean.head(3)

,order_id,order_hour_of_day
0,2539329,8
1,2398795,7
2,473747,12


In [0]:
#Merge the two sets
prod_ord_time = pd.merge(products_ordered, orders_clean, how='left', on='order_id')
prod_ord_time.sample(5)

,order_id,product_id,product_name,order_hour_of_day
32187763,3395076,22227,Coconut Cream Pie,16
9158117,966711,45646,Grated Parmesan Cheese,9
16359050,1725832,39275,Organic Blueberries,8
4547475,480004,21616,Organic Baby Arugula,14
10832123,1143578,21278,All Purpose Crushed Tomatoes,1


In [0]:
#Create condition to identify night items
#popular is >2,900
print(prod_ord_time.shape)
conditioned_ordertimes = prod_ord_time[prod_ord_time['order_hour_of_day'] > 12]

(33819106, 4)


In [0]:
#Build top 25 list
conditioned_ordertimes['product_name'].value_counts().head(25)

Banana                      275720
Bag of Organic Bananas      221476
Organic Strawberries        157296
Organic Baby Spinach        143816
Organic Hass Avocado        126847
Organic Avocado             106522
Large Lemon                  90751
Limes                        84053
Strawberries                 82881
Organic Whole Milk           80125
Organic Raspberries          79601
Organic Yellow Onion         66821
Organic Garlic               65612
Organic Zucchini             62881
Organic Blueberries          59432
Cucumber Kirby               57974
Organic Lemon                52313
Organic Grape Tomatoes       51014
Organic Fuji Apple           50205
Organic Cucumber             49145
Apple Honeycrisp Organic     48111
Seedless Red Grapes          48068
Honeycrisp Apple             46281
Organic Baby Carrots         44692
Yellow Onions                44090
Name: product_name, dtype: int64

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####